In [1]:
import pyodbc
from import_function import import_scoring_data, import_data 
from tto_pricing_features_2 import tto_pricing_features 
# from tto_pricing_features import tto_pricing_features
ct='New'
response='TOTAL_REVENUE_CY'
# TOTAL_REVENUE_CY,AGE_SPOUSE,	AGE_TAXPAYER,	AMOUNT_CHILD_CREDIT,	AMOUNT_EITC,	AMOUNT_HOPE_CREDIT,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	CHANNEL,	CUSTOMER_DEFINITION_ADJ,	ENTRY_PAGE_GROUP,	FED_FORM_TYPE,	FILING_STATUS,	FLAG_ITEMIZED_DEDUCTIONS,	FSCHA_FLAG,	IMPORT_TYPE,	MAX_FLAG,	MINDBENDER_FLAG,	MISC1099_FLAG,	NEAUTH_DEVICE_TYPE,	NON_CA_REFUND_TRANSFER_FLAG,	NUM_CARE_CONTACTS,	POST_TOTAL_VOTES,	PRE_TOTAL_VOTES,	PRODUCT_EDITION_DESCRIPTION,	PRODUCT_ROLLUP,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	student_taxpayer,	VAUTH_DEVICE_TYPE
#   and CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX'
cont_features, bool_features, catag_features, cont_score_features, bool_score_features, catag_score_features=tto_pricing_features(ct) 
scoring_data = "SELECT A.* FROM (SELECT * FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2015 AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX' )A inner join (SELECT DISTINCT CUSTOMER_KEY FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2015 AND TAX_DAY=120) B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY limit 100" 
data = "SELECT TOTAL_REVENUE_CY,AGE_DEPENDENT_AVG_CY,	AGE_SPOUSE_CY,	AGE_TAXPAYER_CY,	AMOUNT_CHILD_CARE_CREDIT,	AMOUNT_EITC,	AMOUNT_HOPE_CREDIT_CY,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_CY,	auth_id_age_role_CY,	FLAG_OLD_OR_BLIND_CY,	MAX_REVENUE,	MINDBENDER_REVENUE,	NON_CA_REFUND_TRANSFER_REVENUE,	NUM_EXEMPTIONS_CY,	ORDER_AMOUNT,	PRS_SCORE_CY,	PS_REVENUE,	SESSIONS_TO_COMPLETE,	STATE_ATTACH_COUNT,	STATE_REVENUE,	TAX_DAY,	AGE_SPOUSE,	AGE_TAXPAYER,	AMOUNT_CHILD_CREDIT_CY,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	COST_PER_CHANNEL_DAY,	NUM_DEPENDENTS_CY,	NUM_EXEMPTIONS,	NUM_W2,	start_sku_rollup_id_CY,	AGE_DEPENDENT_MAX,	AMOUNT_CHILD_CARE_CREDIT_CY,	AMOUNT_CHILD_CREDIT,	AMOUNT_EITC_CY,	AMOUNT_SOCIAL_SEC,	AMOUNT_TOTAL_INCOME,	BUS_ACCOUNTING_METHOD_CASH,	NUM_DEPENDENTS,	TAX_WEEK FROM CTG_ANALYTICS_WS.SM_PRICING_MODEL WHERE customer_type_rollup='Existing'  order by random() limit 3000000"
df_no_pca = import_data(data, cont_features, bool_features,response, catag_features)
# scoring_df=import_scoring_data(scoring_data, cont_score_features, bool_score_features,catag_score_features)
print df_no_pca

['TOTAL_REVENUE_CY', 'AGE_DEPENDENT_AVG_CY', 'AGE_SPOUSE_CY', 'AGE_TAXPAYER_CY', 'AMOUNT_CHILD_CARE_CREDIT', 'AMOUNT_EITC', 'AMOUNT_HOPE_CREDIT_CY', 'AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_CY', 'auth_id_age_role_CY', 'FLAG_OLD_OR_BLIND_CY', 'MAX_REVENUE', 'MINDBENDER_REVENUE', 'NON_CA_REFUND_TRANSFER_REVENUE', 'NUM_EXEMPTIONS_CY', 'ORDER_AMOUNT', 'PRS_SCORE_CY', 'PS_REVENUE', 'SESSIONS_TO_COMPLETE', 'STATE_ATTACH_COUNT', 'STATE_REVENUE', 'TAX_DAY', 'AGE_SPOUSE', 'AGE_TAXPAYER', 'AMOUNT_CHILD_CREDIT_CY', 'AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION', 'COST_PER_CHANNEL_DAY', 'NUM_DEPENDENTS_CY', 'NUM_EXEMPTIONS', 'NUM_W2', 'start_sku_rollup_id_CY', 'AGE_DEPENDENT_MAX', 'AMOUNT_CHILD_CARE_CREDIT_CY', 'AMOUNT_CHILD_CREDIT', 'AMOUNT_EITC_CY', 'AMOUNT_SOCIAL_SEC', 'AMOUNT_TOTAL_INCOME', 'BUS_ACCOUNTING_METHOD_CASH', 'NUM_DEPENDENTS', 'TAX_WEEK']
['AGE_DEPENDENT_AVG_CY', 'AGE_SPOUSE_CY', 'AGE_TAXPAYER_CY', 'AMOUNT_CHILD_CARE_CREDIT', 'AMOUNT_EITC', 'AMOUNT_HOPE_CREDIT_CY', 'AMOUNT_STUDENT_LOAN_INT

c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


['AGE_DEPENDENT_AVG_CY', 'AGE_SPOUSE_CY', 'AGE_TAXPAYER_CY', 'AMOUNT_CHILD_CARE_CREDIT', 'AMOUNT_EITC', 'AMOUNT_HOPE_CREDIT_CY', 'AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_CY', 'auth_id_age_role_CY', 'FLAG_OLD_OR_BLIND_CY', 'MAX_REVENUE', 'MINDBENDER_REVENUE', 'NON_CA_REFUND_TRANSFER_REVENUE', 'NUM_EXEMPTIONS_CY', 'ORDER_AMOUNT', 'PS_REVENUE', 'SESSIONS_TO_COMPLETE', 'STATE_ATTACH_COUNT', 'STATE_REVENUE', 'AGE_SPOUSE', 'AGE_TAXPAYER', 'AMOUNT_CHILD_CREDIT_CY', 'AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION', 'COST_PER_CHANNEL_DAY', 'NUM_DEPENDENTS_CY', 'NUM_EXEMPTIONS', 'NUM_W2', 'AGE_DEPENDENT_MAX', 'AMOUNT_CHILD_CARE_CREDIT_CY', 'AMOUNT_CHILD_CREDIT', 'AMOUNT_EITC_CY', 'AMOUNT_SOCIAL_SEC', 'AMOUNT_TOTAL_INCOME', 'BUS_ACCOUNTING_METHOD_CASH', 'NUM_DEPENDENTS', 'TOTAL_REVENUE_CY']


import_function.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool['paid_something'] = 0
import_function.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool['paid_something'][df[response] > 0]=1
c:\python27\lib\site-packages\pandas\core\generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\python27\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A 

df_cont done
data_scaled done
just_dummies done
df_trans done
new feature list done
         FLAG_OLD_OR_BLIND_CY  paid_something  TAX_DAY  PRS_SCORE_CY_0.0  \
0                         0.0               1      119               0.0   
1                         0.0               1       74               0.0   
2                         0.0               0      119               0.0   
3                         0.0               1       67               0.0   
4                         0.0               1       73               0.0   
5                         0.0               1      131               0.0   
6                         0.0               0       95               0.0   
7                         0.0               1       72               0.0   
8                         0.0               1       86               0.0   
9                         2.0               1      136               0.0   
10                        0.0               1       69               0.0   
11  

In [2]:
from bin_cont_features import bin_pca
j='True'
response='paid_something'
df_pca=df_no_pca
dummy_pca, length_dict = bin_pca(df_pca,response, j)
print dummy_pca


['<=0.5', '>0.5', 'between 0.5 and 1.5', '>=1.5']
['<=96.5', '<=76.5', 'between 76.5 and 96.5', '>96.5', 'between 96.5 and 131.5', '>=131.5']
['<=0.5', '>0.5', 'between 0.5 and 5.5', '>=5.5']
['<=27.5', '<=24.5', 'between 24.5 and 27.5', '>27.5', 'between 27.5 and 31.5', '>=31.5']
['<=26.5', '<=20.5', 'between 20.5 and 26.5', '>26.5', 'between 26.5 and 32.5', '>=32.5']
['<=0.5', '>0.5', 'between 0.5 and 964.5', '>=964.5']
['<=0.5', '>0.5', 'between 0.5 and 503.5', '>=503.5']
['<=937.5', '<=600.5', 'between 600.5 and 937.5', '>937.5', 'between 937.5 and 1029.5', '>=1029.5']
['<=0.5', '>0.5', 'between 0.5 and 959.5', '>=959.5']
['<=4.5', '<=2.5', 'between 2.5 and 4.5', '>4.5', 'between 4.5 and 9.5', '>=9.5']
['<=5.0', '>5.0', 'between 5.0 and 42.490001678466797', '>=42.490001678466797']
['<=9.994999885559082', '>9.994999885559082', 'between 9.994999885559082 and 27.489999771118164', '>=27.489999771118164']
['<=13.170000076293945', '>13.170000076293945', 'between 13.170000076293945 and 33

In [3]:
ct='New'
response='paid_something'
# print dummy_pca.dtypes
from feature_clustering import feature_clustering
i='pca'
df_no_pca=dummy_pca
df_no_pca, y, plsca= feature_clustering(df_no_pca,response, i)

print df_no_pca, y




(300000, 175)
{80: 84.822636713396619, 50: 84.989829244408568, 60: 84.94403854468915, 90: 84.797061137148788, 70: 84.880660038667145}
50
         COST_PER_CHANNEL_DAY_pcl_'>386155.1875'  \
0                                            0.0   
1                                            0.0   
2                                            0.0   
3                                            0.0   
4                                            1.0   
5                                            0.0   
6                                            0.0   
7                                            0.0   
8                                            0.0   
9                                            0.0   
10                                           0.0   
11                                           0.0   
12                                           0.0   
13                                           0.0   
14                                           0.0   
15                             

In [4]:
from clustering import obs_clustering
clust='False'
df=df_no_pca
print df.shape, list(df)

df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)
# print df.shape, df_w_labels.shape, df_w_clust.shape

(3000000, 42) ["COST_PER_CHANNEL_DAY_pcl_'>386155.1875'", "AGE_TAXPAYER_CY_pcl_'<=26.5'", "AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_pcl_'>=999.5'", "AMOUNT_EITC_pcl_'>=503.5'", "MAX_REVENUE_pcl_'between 5.0 and 42.490001678466797'", "AGE_TAXPAYER_CY_pcl_'between 20.5 and 26.5'", "TAX_DAY_pcl_'between 96.5 and 131.5'", "AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_CY_pcl_'between 0.5 and 959.5'", "AGE_TAXPAYER_CY_pcl_'<=20.5'", "AGE_SPOUSE_CY_pcl_'<=27.5'", "AGE_TAXPAYER_pcl_'<=23.5'", "AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_CY_pcl_'<=0.5'", "AGE_TAXPAYER_pcl_'between 27.5 and 32.5'", "auth_id_age_role_CY_pcl_'>4.5'", "AMOUNT_EITC_CY_pcl_'<=0.5'", "AGE_TAXPAYER_pcl_'>27.5'", "AMOUNT_EITC_pcl_'<=0.5'", "AMOUNT_EITC_pcl_'>0.5'", "AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_pcl_'between 0.5 and 999.5'", "auth_id_age_role_CY_pcl_'<=4.5'", "auth_id_age_role_CY_pcl_'between 2.5 and 4.5'", "AGE_SPOUSE_CY_pcl_'between 24.5 and 27.5'", "AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_pcl_'>0.5'", "AMOUNT_CHILD_CREDIT

In [5]:
from pca_dim_red import pca_code
i='False'
response='paid_something'
df_no_pca=df

df_pca, fitted_pca = pca_code(df_no_pca,response,i)
print df_pca



         COST_PER_CHANNEL_DAY_pcl_'>386155.1875'  \
0                                            0.0   
1                                            0.0   
2                                            0.0   
3                                            0.0   
4                                            1.0   
5                                            0.0   
6                                            0.0   
7                                            0.0   
8                                            0.0   
9                                            0.0   
10                                           0.0   
11                                           0.0   
12                                           0.0   
13                                           0.0   
14                                           0.0   
15                                           0.0   
16                                           0.0   
17                                           0.0   
18          

In [6]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df_pca
x, y = get_arrays(dummy_pca, response, i, j)
print x.shape
print x.shape
from sklearn.feature_selection import SelectPercentile, f_classif
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
selector = SelectPercentile(f_classif, percentile=0.5)
selector.fit(x, y)
print selector.pvalues_
df_pvalues=pd.DataFrame(selector.pvalues_,columns=['p_value'], index=list(x))
sig_features=df_pvalues[df_pvalues['p_value'] <=0.05]
sig_features.reset_index(['index'], inplace=True)
sig_features.drop(['p_value'], axis=1, inplace=True)
print sig_features


x=x[sig_features['index'].tolist()]
print x.shape


(3000000, 41)
(3000000, 41)
[  4.29819323e-202   0.00000000e+000   0.00000000e+000   7.98441291e-049
   0.00000000e+000   0.00000000e+000   0.00000000e+000   3.56506280e-284
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   2.16434013e-009   0.00000000e+000
   1.21472412e-144   1.31645572e-137   5.24964307e-168   0.00000000e+000
   0.00000000e+000   0.00000000e+000   5.52722022e-204   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   2.21927386e-150
   0.00000000e+000   0.00000000e+000   2.40715617e-163   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000]
                                                index
0             COST_PER_CHANNEL_DAY_pcl_'>386155.1875'
1                        AGE_TAXPAYER_CY_pcl_'<=26.5'
2   AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION_pcl_'>=...
3     

In [8]:
from model_algorithm import algorithm
print x.shape
models, name = algorithm(x, y, response)

(3000000, 41)
Decision_Tree 0.908159844736 0.835157422526 0.852274458157 0.788873021203 218076 75594 99949 506381
Random_Forest 0.898971810713 0.855920043541 0.850403145526 0.765827355853 198443 95227 87360 518970
                                                         sig
ORDER_AMOUNT_pcl_'<=4.75'                           0.535371
NON_CA_REFUND_TRANSFER_REVENUE_pcl_'>=33.165000...  0.094589
AGE_TAXPAYER_CY_pcl_'<=26.5'                        0.080324
AGE_SPOUSE_CY_pcl_'<=27.5'                          0.070245
AGE_TAXPAYER_pcl_'<=23.5'                           0.045859
auth_id_age_role_CY_pcl_'<=4.5'                     0.030017
AGE_TAXPAYER_CY_pcl_'between 20.5 and 26.5'         0.026647
AGE_TAXPAYER_CY_pcl_'<=20.5'                        0.018048
AGE_SPOUSE_CY_pcl_'between 24.5 and 27.5'           0.013404
ORDER_AMOUNT_pcl_'between 4.75 and 39.794998168...  0.013121
auth_id_age_role_CY_pcl_'>4.5'                      0.011898
AGE_SPOUSE_pcl_'between 24.5 and 26.5'              0.

MemoryError: 

In [ ]:
import pandas as pd
import numpy as np
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
defection=pd.DataFrame()
i, j, k=['False','True','False' ]
df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
scoring_df_trans = bin_pca_score_set(df_trans_pca, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c)



scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)

final_scoring_df=pd.concat([zeros_df,scoring_df_trans ], axis=1)
final_scoring_df.set_index('CUSTOMER_KEY', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)
x_score.set_index('CUSTOMER_KEY', inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()


In [ ]:
from data_intersection import scoring_data_intersection
data = scoring_data_intersection(df_no_pca, scoring_df)
# print data.head()
print df_cont